# Импорт всех библиотек

In [11]:
import json
import math
import os
import time
from functools import partial
from urllib.parse import urlencode

import faiss
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import requests
from catboost import CatBoostClassifier, CatBoostRanker, Pool
from faiss import read_index, write_index
from sentence_transformers import SentenceTransformer
from progress.bar import IncrementalBar

# Подгружаем модель "ranker.ckpt"

In [2]:
model = CatBoostRanker()
model.load_model("ranker.ckpt")

# Загружаем индексы faiss

In [3]:
index = read_index("candidates.index")
index.is_trained, index.ntotal

(True, 7000000)

# Подгружаем "ind2videoid.json"

In [4]:
with open("ind2videoid.json", "r") as f:
    ind2videoid = json.load(f)

# Подгружаем тестовый датасет

In [5]:
import csv

queries = []
with open("test_dataset_submission_queries.csv", mode="r", newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        queries.append(row["query"])

# Загружаем модель SentenceTransformer

In [6]:
st_model = SentenceTransformer("cointegrated/rubert-tiny2", device="cuda")

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

.gitignore:   0%|          | 0.00/6.00 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

tinybert-ru-labse-adapter-v2.pt:   0%|          | 0.00/963k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [7]:
videos_df = pd.read_parquet(
    "videos.parquet", engine="fastparquet", columns=["video_id", "video_title"]
)

In [10]:
%%time
def search_videos(query, topk=5):
    query_embedding = st_model.encode([query])[0]
    _, faiss_ind = index.search(np.array([query_embedding]), topk)

    videos_ids = [ind2videoid[f"{id}"] for id in faiss_ind[0]]

    videos_info = videos_df[videos_df["video_id"].isin(videos_ids)]

    return videos_info

i = 0

with open('submission.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f, delimiter =';')
    writer.writerow(["query", "video_id"])
    for query in queries:
        i += 1
        print(i)
        videos = search_videos(query, topk=5).to_numpy()
        for video in videos:
            writer.writerow([query, video[0]])


KeyboardInterrupt: 